# [Kerasチューナーの紹介](https://www.tensorflow.org/tutorials/keras/keras_tuner)

Keras Tunerは最適なハイパーパラメータを選択する（*hyperparameter tuning* または *hypertuning*）ためのライブラリ。

ハイパーパラメータは学習をコントロールする変数であり、学習プロセスにおいて不変である。ハイパーパラメータには二つの種類がある：

- モデルハイパーパラメータ（model hyperparameters）：隠れ層やユニット数などのモデルを決定づけるパラメータ
- アルゴリズムハイパーパラメータ（algorithm hyperparameters）：学習率などの学習のスピードや質に影響するパラメータ

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
import kerastuner as kt

Keras Tunerを使って最適なハイパーパラメータを見つける。ここではFashion MNISTデータセットを使って、服の画像データを分類する。

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

ハイパーチューニングを行うには、モデルの構造に加えてハイパーパラメータの探索空間も定義する必要がある。ハイパーチューニングを行うためのモデルを**ハイパーモデル**と呼ぶ。ハイパーモデルは以下のいづれかの方法で定義できる。

- モデル構築関数を使う
- APIのクラスである`HyperModel`をサブクラス化する

視覚的に行うために事前に定義された`HyperModel`クラスである、[HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperxception-class)や[HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class)を使うこともできる。

ここではモデル構築関数を使う。この関数はコンパイルされたモデルを返す。

In [7]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32 - 512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    
    return model

ハイパーモデルをインスタンス化する。Keras Tunerには４つのモデル（`RandomSearch`, `Hyperband`, `BayesianOptimization`, `Sklearn`）がある。ここでは`Hyperband`チューナーを利用する。

In [8]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

Hyperbandチューニングでは、たくさんのモデルを小さいエポックで訓練し、パフォーマンスがより良かった上位半分のモデルを次のラウンドに出場させる。ハイパーパラメータの探索を始める前に、コールバックを定義して、各トレーニングステップごとに出力をクリアするようにする。

In [9]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

ハイパーパラメータ探索を実行する。

In [10]:
tuner.search(
    img_train,
    label_train,
    epochs=10,
    validation_data=(img_test, label_test),
    callbacks=[ClearTrainingOutput()]
)

Trial 30 Complete [00h 00m 21s]
val_accuracy: 0.8666999936103821

Best val_accuracy So Far: 0.8894000053405762
Total elapsed time: 00h 06m 50s
INFO:tensorflow:Oracle triggered exit


AttributeError: 'Hyperband' object has no attribute 'get_best_hyperprarameters'

In [11]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 224 and the optimal learning rate for the optimizer
is 0.001.



ハイパーパラメータのチューニングが終わったら、ついにモデルの学習である。

In [12]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs=10, validation_data=(img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.6055 - accuracy: 0.7876 - val_loss: 0.4090 - val_accuracy: 0.8554
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3727 - accuracy: 0.8646 - val_loss: 0.4023 - val_accuracy: 0.8530
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3326 - accuracy: 0.8775 - val_loss: 0.3744 - val_accuracy: 0.8626
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3048 - accuracy: 0.8884 - val_loss: 0.4107 - val_accuracy: 0.8530
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2877 - accuracy: 0.8933 - val_loss: 0.3640 - val_accuracy: 0.8682
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2711 - accuracy: 0.8990 - val_loss: 0.3458 - val_accuracy: 0.8777
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2584 - accuracy: 0.9052 - val_loss: 0.3219 - val_accuracy:

チューナーをインスタンス化した時に設定した`my_dir/intro_to_kit`ディレクトリには、ハイパーパラメータ探索中に出力されたログやチェックポイントの詳細が保存されている。再度ハイパーチューニングを行なった場合には、このディレクトリから現在の状態を読み込み、探索が途中から再開される。始めからチューニングを行いたい場合は、チューナーをインスタンス化する時に`overwrite=True`とする。

まとめ

最適なハイパーパラメータを見つけるための、Keras Tunerの使い方について紹介した。より詳しく知りたい場合は：
- [Keras Tuner on the TensorFlow blog](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
- [Keras Tuner website](https://keras-team.github.io/keras-tuner/)

また、インタラクティブにハイパーパラメータを調節したい場合には、TensorBoardにある[HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams)を参考にする。